# Descripción del proyecto
Trabajas en la compañía de extracción de petróleo OilyGiant. Tu tarea es encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo.

Para completar esta tarea, tendrás que realizar los siguientes pasos:

1. Leer los archivos con los parámetros recogidos de pozos petrolíferos en la región seleccionada: calidad de crudo y volumen de reservas.
2. Crear un modelo para predecir el volumen de reservas en pozos nuevos.
3. Elegir los pozos petrolíferos que tienen los valores estimados más altos.
4. Elegir la región con el beneficio total más alto para los pozos petrolíferos seleccionados.

Tienes datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. Crea un modelo que ayude a elegir la región con el mayor margen de beneficio. Analiza los beneficios y riesgos potenciales utilizando la técnica bootstrapping.



# Condiciones:
Solo se debe usar la regresión lineal para el entrenamiento del modelo.
Al explorar la región, se lleva a cabo un estudio de 500 puntos con la selección de los mejores 200 puntos para el cálculo del beneficio.
El presupuesto para el desarrollo de 200 pozos petroleros es de 100 millones de dólares.
Un barril de materias primas genera 4.5 USD de ingresos. El ingreso de una unidad de producto es de 4500 dólares (el volumen de reservas está expresado en miles de barriles).

# Descripción de datos
Los datos de exploración geológica de las tres regiones se almacenan en archivos:

1. geo_data_0.csv. Descarga el conjunto de datos
2. geo_data_1.csv. Descarga el conjunto de datos
3. geo_data_2.csv. Descarga el conjunto de datos
4. id — identificador único de pozo de petróleo
5. f0, f1, f2 — tres características de los puntos (su significado específico no es importante, pero las características en sí son significativas)
6. product — volumen de reservas en el pozo de petróleo (miles de barriles).

Instrucciones del proyecto
        
        1. Parte I
1. Descarga y prepara los datos. Explica el procedimiento.
2. Entrena y prueba el modelo para cada región en geo_data_0.csv:
    3. Divide los datos en un conjunto de entrenamiento y un conjunto de validación en una proporción de 75:25
    4. Entrena el modelo y haz predicciones para el conjunto de validación.
    5. Guarda las predicciones y las respuestas correctas para el conjunto de validación.
    6. Muestra el volumen medio de reservas predicho y RMSE del modelo.
    7. Analiza los resultados.

8. Coloca todos los pasos previos en funciones, realiza y ejecuta los pasos 2.1-2.5 para los archivos 'geo_data_1.csv' y 'geo_data_2.csv'.

        2. Parte II
9. Prepárate para el cálculo de ganancias:

    10. Almacena todos los valores necesarios para los cálculos en variables separadas.

    11. Dada la inversión de 100 millones por 200 pozos petrolíferos, de media un pozo petrolífero debe producir al menos un valor de 500,000 dólares en unidades para evitar pérdidas (esto es equivalente a 111.1 unidades). Compara esta cantidad con la cantidad media de reservas en cada región.

    12. Presenta conclusiones sobre cómo preparar el paso para calcular el beneficio.

13. Escribe una función para calcular la ganancia de un conjunto de pozos de petróleo seleccionados y modela las predicciones:

    14. Elige los 200 pozos con los valores de predicción más altos de cada una de las 3 regiones (es decir, archivos 'csv').

    15. Resume el volumen objetivo de reservas según dichas predicciones. Almacena las predicciones para los 200 pozos para cada una de las 3 regiones.

    16. Calcula la ganancia potencial de los 200 pozos principales por región. Presenta tus conclusiones: propón una región para el desarrollo de pozos petrolíferos y justifica tu elección.

17. Calcula riesgos y ganancias para cada región:

    18. Utilizando las predicciones que almacenaste en el paso 4.2, emplea la técnica del bootstrapping con 1000 muestras para hallar la distribución de los beneficios.

    19. Encuentra el beneficio promedio, el intervalo de confianza del 95% y el riesgo de pérdidas. La pérdida es una ganancia negativa, calcúlala como una probabilidad y luego exprésala como un porcentaje.

    20. Presenta tus conclusiones: propón una región para el desarrollo de pozos petrolíferos y justifica tu elección. ¿Coincide tu elección con la elección anterior en el punto 4.3?

# Parte I

       1. Parte I
1. Descarga y prepara los datos. Explica el procedimiento.
2. Entrena y prueba el modelo para cada región en geo_data_0.csv:
    3. Divide los datos en un conjunto de entrenamiento y un conjunto de validación en una proporción de 75:25
    4. Entrena el modelo y haz predicciones para el conjunto de validación.
    5. Guarda las predicciones y las respuestas correctas para el conjunto de validación.
    6. Muestra el volumen medio de reservas predicho y RMSE del modelo.
    7. Analiza los resultados.

8. Coloca todos los pasos previos en funciones, realiza y ejecuta los pasos 2.1-2.5 para los archivos 'geo_data_1.csv' y 'geo_data_2.csv'.


In [1]:
##Librerías
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score,precision_score,f1_score
from matplotlib import pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

In [2]:
##Cargar los Datos  
geo0_df = pd.read_csv(r"/home/jovyan/work/geo_data_0.csv")
geo1_df = pd.read_csv(r"/home/jovyan/work/geo_data_1.csv")
geo2_df = pd.read_csv(r"/home/jovyan/work/geo_data_2.csv")

In [3]:
##Visualizar los datos (No hay ausentes)
#geo0_df.info()
#geo1_df.info()
#geo2_df.info()

In [4]:
##Ver la naturaleza de algún df
geo0_df.head(5)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


## Entrena y prueba el modelo para cada región en geo_data_0.csv:

Regresión Lineal
Se busca que los parámetros MSE y RMSE sean lo más bajos posible, ya que indican menor error en las predicciones. Se recomienda usar el cual no toma el promedio del dataframe target.

## Divide los datos en un conjunto de entrenamiento y un conjunto de validación en una proporción de 75:25

Geo 0

In [5]:
##Función para poder dividir el conjunto de datos en validación y entrenamiento
def evaluation(geo):
    ##Caraterísticas y objetivo
    features = geo.drop(['id','product'],axis=1)
    target = geo['product']

    features_train,features_valid,target_train, target_valid = train_test_split(features,target,test_size=0.25,random_state=12345)

    ##Modelo de Regresión Lineal
    model = LinearRegression()
    model.fit(features_train,target_train)

    ##Predicciones
    predicted_valid1 = model.predict(features_valid)
    mse = mean_squared_error(target_valid, predicted_valid1)
    mae = mean_absolute_error(target_valid, predicted_valid1)
    
    ##Score
    print('Score')
    print(model.score(features_valid, target_valid))

    ##MSE, RMSE, MAE métricas para regresión
    print('\n')
    print('Linear Regression')
    print('MSE =', mse)
    print('RMSE =', mse ** 0.5)
    print ('\n')
    print('MAE =',mae )

    print('\n')
    
    ##MSE
    predicted_valid = pd.Series(target_train.mean(), index=target_valid.index)
    mse = mean_squared_error(target_valid, predicted_valid)
    print('Mean')
    print('MSE =', mse)
    print('RMSE =', mse ** 0.5)
    return (predicted_valid1)

In [6]:
predicted_geo0 = evaluation(geo0_df)

Score
0.27994321524487786


Linear Regression
MSE = 1412.2129364399243
RMSE = 37.5794217150813


MAE = 30.919600777151313


Mean
MSE = 1961.5678757223516
RMSE = 44.289591053907365


Geo 1

In [7]:
predicted_geo1 = evaluation(geo1_df)

Score
0.9996233978805127


Linear Regression
MSE = 0.7976263360391157
RMSE = 0.893099286775617


MAE = 0.7187662442124758


Mean
MSE = 2117.9734309299147
RMSE = 46.02144533725462


Geo 2

In [8]:
predicted_geo2 = evaluation(geo2_df)

Score
0.20524758386040443


Linear Regression
MSE = 1602.3775813236196
RMSE = 40.02970873393434


MAE = 32.792652105481814


Mean
MSE = 2016.2210072435087
RMSE = 44.90234968510566


In [9]:
print(predicted_geo2)
print(type(predicted_geo2))

[ 93.59963303  75.10515854  90.06680936 ...  99.40728116  77.77991248
 129.03241718]
<class 'numpy.ndarray'>


# Parte II

4. Prepárate para el cálculo de ganancias:

    10. Almacena todos los valores necesarios para los cálculos en variables separadas.

    11. Dada la inversión de 100 millones por 200 pozos petrolíferos, de media un pozo petrolífero debe producir al menos un valor de 500,000 dólares en unidades para evitar pérdidas (esto es equivalente a 111.1 unidades). Compara esta cantidad con la cantidad media de reservas en cada región.

    12. Presenta conclusiones sobre cómo preparar el paso para calcular el beneficio.

5. Escribe una función para calcular la ganancia de un conjunto de pozos de petróleo seleccionados y modela las predicciones:

    14. Elige los 200 pozos con los valores de predicción más altos de cada una de las 3 regiones (es decir, archivos 'csv').

    15. Resume el volumen objetivo de reservas según dichas predicciones. Almacena las predicciones para los 200 pozos para cada una de las 3 regiones.

    16. Calcula la ganancia potencial de los 200 pozos principales por región. Presenta tus conclusiones: propón una región para el desarrollo de pozos petrolíferos y justifica tu elección.

6. Calcula riesgos y ganancias para cada región:

    18. Utilizando las predicciones que almacenaste en el paso 4.2, emplea la técnica del bootstrapping con 1000 muestras para hallar la distribución de los beneficios.

    19. Encuentra el beneficio promedio, el intervalo de confianza del 95% y el riesgo de pérdidas. La pérdida es una ganancia negativa, calcúlala como una probabilidad y luego exprésala como un porcentaje.

    20. Presenta tus conclusiones: propón una región para el desarrollo de pozos petrolíferos y justifica tu elección. ¿Coincide tu elección con la elección anterior en el punto 4.3?

In [10]:
##Volumen de barriles de petroleo y ganancia por barril
def mean_geo (geo):
    geo_mean_p =geo['product'].mean()
    print('Volumen promedio de reservas',geo['product'].mean())
    print('Ganancia promedio por barril',geo_mean_p*5000)
    return

In [11]:
print(predicted_geo0)

[ 95.89495185  77.57258261  77.89263965 ...  61.50983303 118.18039721
 118.16939229]


El Volumen promedio por región está por debajo de las 111 unidades mínimas para que la inversión sea recuperada dentro de los pozos, es decir, que la ganancia es menor a los 500,000 esperados para evitar pérdidas. La región con los numeros más altos es la región geo 2. Tiene 95 barriles por pozo y una ganacia de 475,000.

In [12]:
mean_geo(geo0_df)

Volumen promedio de reservas 92.50000000000001
Ganancia promedio por barril 462500.00000000006


In [13]:
mean_geo(geo1_df)

Volumen promedio de reservas 68.82500000000002
Ganancia promedio por barril 344125.00000000006


In [14]:
mean_geo(geo2_df)

Volumen promedio de reservas 95.00000000000004
Ganancia promedio por barril 475000.00000000023


Se elabora una función para poder visualizar la preddicción de los 200 pozos con mayor número estimado de barriles producidos, por región. Se visualiza que se elaboran tablas donde el profit general es el beneficio bruto y el total profit es el beneficio operativo. El beneficio operativo se obtiene del beneficio bruto menos la inversión por pozo, mencionada de 500,000.

In [15]:
def top_200(geo_predicted):
    geo_predicted = pd.Series(geo_predicted)
    geo_20= geo_predicted.sort_values(ascending=False).reset_index()
    geo_20.columns=['position','units']
    #geo_20.columns=['id','product']
    ##Ganancia bruta
    geo_20['general profit'] = geo_20['units']*5000
    ##Ganancia operativa 
    geo_20['total profit'] = geo_20['general profit']-500000
    print(geo_20[:200])
    return

Geo 0

In [16]:
top200_geo0=top_200(predicted_geo0)

     position       units  general profit   total profit
0        9317  180.180713   900903.565306  400903.565306
1         219  176.252213   881261.065836  381261.065836
2       10015  175.850623   879253.116417  379253.116417
3       11584  175.658429   878292.144777  378292.144777
4       23388  173.299686   866498.430921  366498.430921
..        ...         ...             ...            ...
195      7888  148.507064   742535.319302  242535.319302
196      7890  148.481767   742408.833413  242408.833413
197     24051  148.476498   742382.490801  242382.490801
198     24160  148.436761   742183.807454  242183.807454
199     20340  148.365941   741829.704999  241829.704999

[200 rows x 4 columns]


Geo 1

In [17]:
top200_geo1=top_200(predicted_geo1)

     position       units  general profit   total profit
0       20430  139.818970   699094.849063  199094.849063
1        7777  139.773423   698867.114787  198867.114787
2        8755  139.703330   698516.651566  198516.651566
3        1178  139.560938   697804.687974  197804.687974
4        4285  139.516754   697583.771467  197583.771467
..        ...         ...             ...            ...
195     12774  138.421423   692107.115595  192107.115595
196      8040  138.416960   692084.800991  192084.800991
197      2257  138.413881   692069.403523  192069.403523
198     17108  138.412834   692064.169031  192064.169031
199      9693  138.410697   692053.483684  192053.483684

[200 rows x 4 columns]


Geo 2

In [18]:
top200_geo2=top_200(predicted_geo2)

     position       units  general profit   total profit
0       22636  165.856833   829284.165872  329284.165872
1       24690  165.679685   828398.426330  328398.426330
2        7811  163.439962   817199.811660  317199.811660
3        1581  162.062589   810312.947497  310312.947497
4        6751  161.797476   808987.381408  308987.381408
..        ...         ...             ...            ...
195      5541  142.490763   712453.814169  212453.814169
196     18126  142.485922   712429.611557  212429.611557
197     11122  142.465777   712328.886137  212328.886137
198     22669  142.454763   712273.816318  212273.816318
199     23447  142.408851   712044.256748  212044.256748

[200 rows x 4 columns]


Se utilizó el método de bootstrapping para estimar la distribución de los ingresos y calcular métricas clave como el ingreso promedio, el intervalo de confianza del 95% y el riesgo de pérdidas.

Se aprecia que los promedios son muy acercados a los promedios reales de ganancia en los pasos anteriores. El pozo que presenta menor riesgo de inversión es la región geo 2 con un porcentaje de 64.4%. Quiere decir que ese es el riesgo de que el pozo invertido produzca menores ingresos que los invertidos y que la ganancia sea negativa

In [19]:
# Función para calcular los ingresos
def revenue(target, probabilities, count):
    # Ordena las probabilidades de mayor a menor
    probs_sorted = probabilities.sort_values(ascending=False)
    # Selecciona los primeros 'count' elementos (10 pozos)
    selected = target[probs_sorted.index][:count]
    return 5000 * selected.mean()

In [20]:
# Función para ganancia promedio, riesgo e intervalo de  confianza
def final (geo,predicted):
    features= geo.drop(['id','product'],axis=1)
    target = geo['product']
    probabilities= pd.Series(predicted)

# Asegurar que target y probabilities tengan los mismos índices
    target= geo['product'].reset_index(drop=True)
    probabilities= probabilities.reset_index(drop=True)

    state = np.random.RandomState(12345)
    values = []

# Realiza el bootstrapping: toma 1000 muestras
    for i in range(1000):
    
        target_subsample = target.sample(n=200, replace=True, random_state=state).reset_index(drop=True)  # Cambié n=1000 a n=25000
        probs_subsample = probabilities[target_subsample.index]
        values.append(revenue(target_subsample, probs_subsample, 10))  # Ajusta el número de pozos (10)

# Convierte los valores de ingresos a una Serie de Pandas
    values = pd.Series(values)
    mean = values.mean()

# Intervalo de Confianza del 95%
    lower = values.quantile(0.025)  # 2.5% cuantil (para el límite inferior)
    upper = values.quantile(0.975)  # 97.5% cuantil (para el límite superior)

# Riesgo de Pérdidas (probabilidad de tener ingresos negativos)
    loss_risk = (values < 500000 ).mean() * 100  # Convertimos la probabilidad en porcentaje

    print(f"Ingresos promedio: {mean}")
    print(f"Intervalo de confianza del 95%: ({lower}, {upper})")
    print(f"Riesgo de pérdidas: {loss_risk}%")
    return

In [21]:
def top200s(predicted):
    geo_predicted = pd.Series(predicted)
    geo_sorted = geo_predicted.sort_values(ascending=False)
    top_200_array = np.array(geo_sorted.iloc[:200])
    print(top_200_array)

    return top_200_array  # Retorna el array si lo necesitas

In [22]:
geo0_predicted=top200s(predicted_geo0)

[180.18071306 176.25221317 175.85062328 175.65842896 173.29968618
 172.62797248 172.50167789 169.12423358 168.78760151 168.20601476
 167.9553527  167.89493172 166.73719173 166.64783024 166.58664229
 166.51253196 165.77868214 164.88665844 164.76765859 164.7464549
 164.60991694 164.51805395 164.10007177 163.63628349 163.35261037
 162.95212231 162.2441628  162.24349816 161.92411048 161.70643654
 161.70054385 161.14661861 160.96571699 160.94719592 160.91870049
 160.56657568 160.49284653 160.35757296 160.31411761 160.19923809
 160.0642034  160.03644529 159.92791076 159.89191394 159.86367774
 159.68067978 159.55030385 159.50773314 159.03061812 158.57697873
 158.57682628 158.49080939 158.11221037 158.1048762  157.90933803
 157.72043411 157.71395384 157.53653905 157.4295083  157.40358913
 157.07172387 156.83431917 156.73182285 156.6966888  156.49858586
 156.47607107 156.42685601 156.41834157 156.36528745 156.36126347
 156.32975573 156.22161471 156.09039845 156.07788946 156.0187806
 155.7717916

In [23]:
geo1_predicted=top200s(predicted_geo1)
print(type(geo1_predicted))

[139.81896981 139.77342296 139.70333031 139.56093759 139.51675429
 139.4729438  139.47205891 139.44650088 139.43165277 139.42452597
 139.38874898 139.30774543 139.25404759 139.25068009 139.15876273
 139.1434329  139.11526578 139.10472303 139.08129975 139.07978165
 139.07014053 139.06671848 139.05518926 139.05255628 139.03526451
 139.02835462 139.02255401 139.00712372 139.0065201  139.0025295
 139.00128309 138.97996329 138.95407219 138.95392375 138.94436831
 138.93790535 138.93784463 138.93609088 138.93382169 138.92497835
 138.90826059 138.90787204 138.90642539 138.89572203 138.89147041
 138.88937789 138.88852548 138.88751906 138.88730277 138.88161812
 138.87033255 138.86281988 138.85962626 138.85914285 138.8392558
 138.83683753 138.8331988  138.82825512 138.82820825 138.8261413
 138.82566959 138.81246323 138.81169436 138.80873071 138.80001932
 138.79860567 138.79454299 138.78957577 138.78917995 138.78287979
 138.78229125 138.76992383 138.76510252 138.76062158 138.74906275
 138.74884502

In [24]:
geo2_predicted=top200s(predicted_geo2)

[165.85683317 165.67968527 163.43996233 162.0625895  161.79747628
 161.63272966 160.90714301 160.79107005 160.47889601 159.99053953
 159.9566195  159.18637626 159.1131953  158.84142132 157.58869037
 157.15664851 156.86005825 156.57696724 156.21517371 156.05944004
 155.99726798 155.84901909 155.84236129 155.76590908 155.66456621
 155.59138885 154.99613618 154.24937687 154.19453774 154.17171937
 153.82903089 153.81311512 153.76961857 153.65059807 153.60866789
 153.30168881 152.80154046 152.54408434 152.51735229 152.23743012
 152.08364982 152.01820434 151.54286075 151.27720085 150.841098
 150.72386796 150.68665259 150.67457991 150.50631759 150.4181502
 150.41550335 149.82337414 149.57955981 149.3790756  149.30798057
 149.29295508 149.11269363 149.09832081 148.99705066 148.87576194
 148.82975017 148.78806918 148.76793597 148.67791392 148.6357326
 148.53967624 148.40775305 148.3638055  148.28408203 148.27492066
 148.26207593 148.22766157 148.21330628 148.09168082 148.05562676
 147.99037938 

In [25]:
final(geo0_df,geo0_predicted)

Ingresos promedio: 464359.2725475618
Intervalo de confianza del 95%: (325172.076379638, 600721.1587273463)
Riesgo de pérdidas: 70.5%


In [26]:
final(geo1_df,geo1_predicted)

Ingresos promedio: 347436.03534041037
Intervalo de confianza del 95%: (198210.13497039955, 496325.29639122984)
Riesgo de pérdidas: 97.7%


In [27]:
final(geo2_df,geo2_predicted)

Ingresos promedio: 473291.8200250828
Intervalo de confianza del 95%: (335645.3051206137, 618345.027469659)
Riesgo de pérdidas: 64.4%


# Conclusión
Se eliminó la columna irrelevante ('id')  y la variable objetivo (target) del los dataframe de las tres regiones de las características (features). El correcto almacenamiento de las predicciones (probabilities) en una pd.Series fue para su procesamiento posteriormente.

Se realizó la técnica de bootstrapping con 1000 muestras y el cálculo de la distribución de ingreso. El ingreso promedio, intervalo de confianza del 95% y riesgo de pérdidas. Se seleccionaron los 10 mejores pozos en cada iteración para maximizar las ganancias. Tomando en cuenta que laa inversión es de 500,000 dólares por pozo y que la ganancia se calcula sobre los 10 mejores pozos seleccionados. Se evaluó el riesgo de pérdidas, es decir, la probabilidad de que los ingresos sean inferiores a 500,000 dólares. Esto junto con el ingreso promedio y el rango de confianza sobre los ingresos es calve para la toma de decisiones.

Geo 2 representa la región que brinda mayores ingresos con un menor riesgo de perdida de inversión, por lo que es la mejor región para invertir de las tres


# Evaluación del proyecto
Hemos definido los criterios de evaluación para el proyecto. Lee esto con atención antes de pasar al ejercicio.

Esto es lo que los revisores buscarán cuando evalúen tu proyecto:

¿Cómo preparaste los datos para el entrenamiento?
¿Seguiste todos los pasos de las instrucciones?
¿Consideraste todas las condiciones del negocio?
¿Cuáles son tus hallazgos sobre el estudio de tareas?
¿Aplicaste correctamente la técnica bootstrapping?
¿Sugeriste la mejor región para el desarrollo de pozos? ¿Justificaste tu elección?
¿Evitaste la duplicación de código?
¿Mantuviste la estructura del proyecto y el código limpio?
Ya tienes tus hojas informativas y los resúmenes de los capítulos, por lo que todo está listo para continuar con el proyecto.

¡Buena suerte!